# Route Identification Over Time

Recent observations shows small chages in routes over time. Specifically in the following fields:
* route ID
* route short name
* route long name
* route desc

Need to observe these route changes in order to account for these changes in future analyses.

## Objective
1. Query data from `fct_monthly_routes` to help identify variences in Routes. Query for 2023, a couple of months. 
2. Save data to GCS `gtfs_schedule` bucket
3. Filter down data to `Sacramento Regional Transit`, identify and observe routes for any variences


## function from open_data/download_vehicle_position.py
    
    import datetime
    import gcsfs
    import geopandas as gpd
    import pandas as pd
    import shapely
    import sys

    from calitp_data_analysis.tables import tbls
    from calitp_data_analysis import utils
    from loguru import logger
    from siuba import *

    from shared_utils import schedule_rt_utils
    
    def download_vehicle_positions(
        date: str,
        operator_names: list
    ) -> pd.DataFrame:    
    
        df = (tbls.mart_gtfs.fct_vehicle_locations()
              >> filter(_.service_date == date)
              >> filter(_.gtfs_dataset_name.isin(operator_names))
              >> select(_.gtfs_dataset_key, _.gtfs_dataset_name,
                        _.schedule_gtfs_dataset_key,
                        _.trip_id, _.trip_instance_key,
                        _.location_timestamp,
                        _.location)
                  >> collect()
             )

In [3]:
#imports

import datetime
import gcsfs
import geopandas as gpd
import pandas as pd
import shapely
import sys

from calitp_data_analysis.tables import tbls
from calitp_data_analysis import utils
from loguru import logger
from siuba import *

from shared_utils import schedule_rt_utils

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


ModuleNotFoundError: No module named 'shared_utils'

In [14]:
# test to query fct_monthly_routes
def get_monthly_routes(
        year: str,
        months: list
    ) -> pd.DataFrame:    
    
        df = (tbls.mart_gtfs.fct_monthly_routes()
              >> filter(_.year == year)
              >> filter(_.month.isin(months))
              >> select(_.key, _.source_record_id,
                        _.name,
                        _.route_id, _.shape_id,
                        _.month,
                        _.year,
                       _.pt_array)
                  >> collect()
             )
        return df

In [25]:
monthly_routes = get_monthly_routes(2023, [3, 4, 5])

/opt/conda/lib/python3.9/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_array'
  sqlalchemy.util.warn(


In [26]:
#peaking into df to make sure everything looks good

#shape shows 11,927 rows and 8 columns
display(monthly_routes.shape)

#type shows data is in df
display(type(monthly_routes))

#columns return all the columns we listed in the function
display(list(monthly_routes.columns))

#value_counts confirm df only has rows from 2023 March to May
display(monthly_routes.value_counts(subset=['year','month']))

(11927, 8)

pandas.core.frame.DataFrame

['key',
 'source_record_id',
 'name',
 'route_id',
 'shape_id',
 'month',
 'year',
 'pt_array']

year  month
2023  5        4180
      3        3899
      4        3848
dtype: int64

In [29]:
monthly_routes.groupby(['name']).route_id.unique()

name
AC Transit Schedule     [19, 706, 657, J, 18, 605, 628, FS, 210, 667, ...
ACE Schedule                                                        [ACE]
Alhambra Schedule                                   [GreenLine, BlueLine]
Amador Schedule                            [570, 569, 660, 661, 573, 571]
Amtrak Schedule         [21314, 85, 75, 70, 39297, 37329, 42896, 40751...
                                              ...                        
YARTS Schedule                                          [580, 1094, 2005]
Yolobus Schedule        [fd4f01d1-f65d-4955-a793-32d619ef0cea, 7e1a17f...
Yuba-Sutter Schedule    [6464, 472, 363, 473, 6465, 369, 362, 367, 364...
Yuma Schedule                        [12, 14, 17, 10, 21, 11, 23, 16, 28]
eTrans Schedule                                                     [277]
Name: route_id, Length: 212, dtype: object

In [44]:
#checking for rows for 'Sacramento Schedule'
monthly_routes[monthly_routes['name'] == 'Sacramento Schedule']

,key,source_record_id,name,route_id,shape_id,month,year,pt_array
105,7637affa883267023c84ff2050e84b27,recbzZQUIdMmFvm1r,Sacramento Schedule,075,45329,5,2023,[]
107,1198d65e124804387e27ac4808ade12e,recbzZQUIdMmFvm1r,Sacramento Schedule,30,45083,3,2023,[]
273,e69b9d9211debc5dd19e34aff1ac91ad,recbzZQUIdMmFvm1r,Sacramento Schedule,255,45457,5,2023,[]
275,d4bfb5f7a5ffd3a13c3f24ea48f1607f,recbzZQUIdMmFvm1r,Sacramento Schedule,103,45357,4,2023,"[POINT(-121.516625 38.48037), POINT(-121.51665..."
314,25c4f5d2b3dbeb22b4b3c27fe8445288,recbzZQUIdMmFvm1r,Sacramento Schedule,519,45496,5,2023,[]
...,...,...,...,...,...,...,...,...
11623,9af2233f4f4a7af1ba90f27c8dbb1d28,recbzZQUIdMmFvm1r,Sacramento Schedule,062,45316,5,2023,[]
11729,3d27bbb7b2a62a0ee4fe43149e7b39fe,recbzZQUIdMmFvm1r,Sacramento Schedule,093,45352,4,2023,"[POINT(-121.38306 38.64513), POINT(-121.38305 ..."
11862,bc9551a2165081abdfe7b2631100baaa,recbzZQUIdMmFvm1r,Sacramento Schedule,001,45169,3,2023,[]
11912,3f38540c4e60d4e017d897a16093d1d8,recbzZQUIdMmFvm1r,Sacramento Schedule,176,45031,3,2023,[]
